In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
import time
import numpy as np
import sklearn

In [2]:
def format_data(df, ticker):
    renamed_columns = {
    "Fecha": 'Date', 
    'Último': f'{ticker}-Close', 
    "Apertura": f'{ticker}-Open', 
    "Máximo": f"{ticker}-High", 
    "Mínimo": f"{ticker}-Low", 
    "Vol.": f"{ticker}-Volume", 
    "% var.": f"{ticker}-% var"
    }
    df = df.rename(columns=renamed_columns)
    df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y")
    return df
    

# OHLC (Open - High - Low - Close) del bitcoin
**Open** -> Precio de apertura

**High** -> Precio máximo

**Low** -> Precio mínimo

**Close** -> Precio de cierre

**Volume** -> Cantidad de BTC que se movieron

In [3]:
startTime = 1502928000000
limit = time.time() * 1000

OHLC_dataframes = []

while (startTime < limit):
    url = 'https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=1d&limit=1000&startTime={startTime}'.format(startTime=startTime)
    res = requests.get(url)
    data = json.loads(res.text)
    df = pd.DataFrame(data, columns=[
        "Open time", "BTC-Open", "BTC-High", "BTC-Low", "BTC-Close", "BTC-Volume",
        "Close time", "Quote asset volume", "Number of trades",
        "Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"
    ])
    df["Date"] = pd.to_datetime(df["Open time"], unit='ms') - pd.Timedelta(days=1)
    startTime = df['Close time'].iloc[-1]
    df = df[["Date", "BTC-Open", "BTC-High", "BTC-Low", "BTC-Close", "BTC-Volume"]]
    df[["BTC-Open", "BTC-High", "BTC-Low", "BTC-Close", "BTC-Volume"]] = df[["BTC-Open", "BTC-High", "BTC-Low", "BTC-Close", "BTC-Volume"]].astype(float)
    OHLC_dataframes.append(df)
    
OHLC_BTC = pd.concat(df for df in OHLC_dataframes)
OHLC_BTC

,Date,BTC-Open,BTC-High,BTC-Low,BTC-Close,BTC-Volume
0,2017-08-16,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-17,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-18,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-19,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-20,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...,...
974,2025-10-07,121332.96,124197.25,121066.14,123306.00,17012.618000
975,2025-10-08,123306.01,123762.94,119651.47,121662.40,21559.360070
976,2025-10-09,121662.41,122550.00,102000.00,112774.50,64171.939270
977,2025-10-10,112774.49,113322.39,109561.59,110644.40,35448.516520


# OHLC del petróleo (WTI) desde la fecha más vieja en la que pudimos obtener datos del BTC
https://es.investing.com/commodities/crude-oil-historical-data

In [4]:
OHLC_WTI = pd.read_csv("CSVs/WTI.csv")
OHLC_WTI = format_data(OHLC_WTI, "WTI")

for feature in OHLC_WTI.columns.values[1:-2]:
    OHLC_WTI[feature] = OHLC_WTI[feature].str.replace(",",".")
OHLC_WTI[["WTI-Close", "WTI-Open", "WTI-High", "WTI-Low"]] = OHLC_WTI[["WTI-Close", "WTI-Open", "WTI-High", "WTI-Low"]].astype(float)

hay_sin_k = OHLC_WTI["WTI-Volume"].str.endswith("K", na=False).all()
if hay_sin_k:
    print("✅ Todos los valores terminan en K")
else:
    print("⚠️ Hay valores que NO terminan en K")

OHLC_WTI.head(25)

⚠️ Hay valores que NO terminan en K


,Date,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,WTI-% var
0,2025-10-05,61.43,61.46,61.79,61.36,"6,92K","0,90%"
1,2025-10-03,60.88,60.70,61.38,60.55,"209,16K","0,66%"
2,2025-10-02,60.48,61.78,62.54,60.40,"290,51K","-2,10%"
3,2025-10-01,61.78,62.46,62.89,61.40,"274,34K","-0,95%"
4,2025-09-30,62.37,63.14,63.26,62.03,"271,65K","-1,70%"
5,2025-09-29,63.45,65.07,65.40,62.98,"294,29K","-2,59%"
6,2025-09-28,65.14,65.00,65.18,64.88,"8,23K","-0,88%"
7,2025-09-26,65.72,65.20,66.42,64.66,"284,99K","1,14%"
8,2025-09-25,64.98,64.80,65.34,64.06,"258,35K","-0,02%"
9,2025-09-24,64.99,63.64,65.05,63.25,"282,72K","2,49%"


In [5]:
print("NaN en la columna WTI-Volume:")
print(OHLC_WTI["WTI-Volume"].isna().sum())
print(OHLC_WTI[OHLC_WTI["WTI-Volume"].isna()])
print("Valores en la columna WTI-Volume sin una K")
filtro = OHLC_WTI["WTI-Volume"].notna() & ~OHLC_WTI["WTI-Volume"].str.endswith("K", na=False)
print(filtro.sum())
print(OHLC_WTI[filtro])

NaN en la columna WTI-Volume:
88
           Date  WTI-Close  WTI-Open  WTI-High  WTI-Low WTI-Volume WTI-% var
27   2025-08-31      63.96     63.98     64.01    63.92        NaN    -0,08%
98   2025-05-25      61.93     61.73     62.15    61.58        NaN     0,65%
282  2024-09-02      73.78     73.00     74.39    72.89        NaN     1,10%
283  2024-09-01      72.98     73.33     73.42    72.97        NaN     0,45%
325  2024-07-04      83.94     83.61     84.20    83.03        NaN     1,11%
...         ...        ...       ...       ...      ...        ...       ...
2041 2018-01-01      60.24     60.26     60.28    60.15        NaN    -0,33%
2046 2017-12-25      58.59     58.41     58.62    58.38        NaN     0,09%
2068 2017-11-23      58.38     57.97     58.58    57.76        NaN     0,62%
2126 2017-09-04      47.41     47.31     47.66    47.16        NaN     0,19%
2127 2017-09-03      47.32     47.31     47.42    47.30        NaN     0,06%

[88 rows x 7 columns]
Valores en la column

In [6]:
def parse_value(x):
    if pd.isna(x):  # mantenemos los NaN por ahora
        return np.nan
    x = str(x).strip()
    
    factor = 1
    if x.endswith("K"):
        factor = 1000
        x = x[:-1]
    elif x.endswith("M"):
        factor = 1000000
        x = x[:-1]
    
    # reemplazamos coma decimal por punto
    x = x.replace(",", ".")
    
    try:
        return float(x) * factor
    except ValueError:
        return np.nan  # en caso de algún valor raro

OHLC_WTI["WTI-Volume"] = OHLC_WTI["WTI-Volume"].apply(parse_value)
OHLC_WTI[OHLC_WTI["WTI-Volume"] >= 1000000]

,Date,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,WTI-% var
1433,2020-04-21,11.57,21.32,22.58,6.50,2290000.0,"-43,37%"
1434,2020-04-20,20.43,24.76,24.92,20.19,1320000.0,"-18,38%"
1454,2020-03-20,22.63,25.59,28.49,22.39,1130000.0,"-12,66%"
1455,2020-03-19,25.91,22.82,28.28,21.77,1190000.0,"24,39%"
1456,2020-03-18,20.83,27.30,27.60,20.52,1000000.0,"-23,78%"
2125,2017-09-05,48.66,47.28,48.98,47.15,1030000.0,"2,64%"


In [7]:
OHLC_WTI.head(25)

,Date,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,WTI-% var
0,2025-10-05,61.43,61.46,61.79,61.36,6920.0,"0,90%"
1,2025-10-03,60.88,60.70,61.38,60.55,209160.0,"0,66%"
2,2025-10-02,60.48,61.78,62.54,60.40,290510.0,"-2,10%"
3,2025-10-01,61.78,62.46,62.89,61.40,274340.0,"-0,95%"
4,2025-09-30,62.37,63.14,63.26,62.03,271650.0,"-1,70%"
5,2025-09-29,63.45,65.07,65.40,62.98,294290.0,"-2,59%"
6,2025-09-28,65.14,65.00,65.18,64.88,8230.0,"-0,88%"
7,2025-09-26,65.72,65.20,66.42,64.66,284990.0,"1,14%"
8,2025-09-25,64.98,64.80,65.34,64.06,258350.0,"-0,02%"
9,2025-09-24,64.99,63.64,65.05,63.25,282720.0,"2,49%"


# OHLC del petróleo (Brent) desde la fecha más vieja en la que pudimos obtener datos del BTC
https://es.investing.com/commodities/brent-oil-historical-data

In [8]:
OHLC_BRENT = pd.read_csv("CSVs/BRENT.csv")
OHLC_BRENT = format_data(OHLC_BRENT, "BRENT")

for feature in OHLC_BRENT.columns.values[1:-2]:
    OHLC_BRENT[feature] = OHLC_BRENT[feature].str.replace(",",".")
OHLC_BRENT[["BRENT-Close", "BRENT-Open", "BRENT-High", "BRENT-Low"]] = OHLC_BRENT[["BRENT-Close", "BRENT-Open", "BRENT-High", "BRENT-Low"]].astype(float)

hay_sin_k = OHLC_BRENT["BRENT-Volume"].str.endswith("K", na=False).all()
if hay_sin_k:
    print("✅ Todos los valores terminan en K")
else:
    print("⚠️ Hay valores que NO terminan en K")

OHLC_BRENT.head(25)

⚠️ Hay valores que NO terminan en K


,Date,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,BRENT-Volume,BRENT-% var
0,2025-10-05,65.10,64.86,65.51,64.84,"5,72K","0,88%"
1,2025-10-03,64.53,64.38,65.02,64.20,"311,95K","0,66%"
2,2025-10-02,64.11,65.45,66.15,64.00,"480,22K","-1,90%"
3,2025-10-01,65.35,66.18,66.57,65.05,"420,60K","-2,49%"
4,2025-09-30,67.02,67.50,67.71,66.84,"19,23K","-1,40%"
5,2025-09-29,67.97,69.50,69.91,67.52,"107,59K","-3,08%"
6,2025-09-26,70.13,69.54,70.76,69.11,"110,38K","1,02%"
7,2025-09-25,69.42,69.04,69.68,68.42,"210,51K","0,16%"
8,2025-09-24,69.31,67.88,69.37,67.51,"222,73K","3,49%"
9,2025-09-23,66.97,65.99,67.42,65.49,"344,39K","1,52%"


In [9]:
print("NaN en la columna BRENT-Volume:")
print(OHLC_BRENT["BRENT-Volume"].isna().sum())
print("Valores en la columna BRENT-Volume sin una K")
filtro = ~OHLC_BRENT["BRENT-Volume"].str.endswith("K", na=False)
print(filtro.sum())
print(OHLC_BRENT[filtro])

NaN en la columna BRENT-Volume:
0
Valores en la columna BRENT-Volume sin una K
1
         Date  BRENT-Close  BRENT-Open  BRENT-High  BRENT-Low BRENT-Volume  \
81 2025-06-13        74.23        70.5        78.5      70.41        1,24M   

   BRENT-% var  
81       7,02%  


In [10]:
OHLC_BRENT["BRENT-Volume"] = OHLC_BRENT["BRENT-Volume"].apply(parse_value)
OHLC_BRENT[OHLC_BRENT["BRENT-Volume"] >= 1000000]

,Date,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,BRENT-Volume,BRENT-% var
81,2025-06-13,74.23,70.5,78.5,70.41,1240000.0,"7,02%"


In [11]:
OHLC_BRENT.head(25)

,Date,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,BRENT-Volume,BRENT-% var
0,2025-10-05,65.10,64.86,65.51,64.84,5720.0,"0,88%"
1,2025-10-03,64.53,64.38,65.02,64.20,311950.0,"0,66%"
2,2025-10-02,64.11,65.45,66.15,64.00,480220.0,"-1,90%"
3,2025-10-01,65.35,66.18,66.57,65.05,420600.0,"-2,49%"
4,2025-09-30,67.02,67.50,67.71,66.84,19230.0,"-1,40%"
5,2025-09-29,67.97,69.50,69.91,67.52,107590.0,"-3,08%"
6,2025-09-26,70.13,69.54,70.76,69.11,110380.0,"1,02%"
7,2025-09-25,69.42,69.04,69.68,68.42,210510.0,"0,16%"
8,2025-09-24,69.31,67.88,69.37,67.51,222730.0,"3,49%"
9,2025-09-23,66.97,65.99,67.42,65.49,344390.0,"1,52%"


# OHLC del SPX (ticker del S&P 500) desde la fecha más vieja en la que pudimos obtener datos del BTC
https://es.investing.com/indices/us-spx-500-historical-data

In [12]:
OHLC_SPX = pd.read_csv("CSVs/SPX.csv")
OHLC_SPX = format_data(OHLC_SPX, "SPX")

OHLC_SPX = OHLC_SPX[['Date', 'SPX-Close', 'SPX-Open', 'SPX-High', 'SPX-Low', 'SPX-% var']]
for feature in OHLC_SPX.columns.values[1:-1]:
    OHLC_SPX[feature] = OHLC_SPX[feature].str.replace(".","")
    OHLC_SPX[feature] = OHLC_SPX[feature].str.replace(",",".")
OHLC_SPX

,Date,SPX-Close,SPX-Open,SPX-High,SPX-Low,SPX-% var
0,2025-10-03,6715.79,6722.14,6750.87,6705.67,"0,01%"
1,2025-10-02,6715.35,6731.31,6731.94,6693.23,"0,06%"
2,2025-10-01,6711.20,6664.92,6718.48,6656.20,"0,34%"
3,2025-09-30,6688.46,6656.19,6691.25,6641.00,"0,41%"
4,2025-09-29,6661.21,6661.58,6677.31,6644.49,"0,26%"
...,...,...,...,...,...,...
2040,2017-08-22,2452.50,2433.80,2454.80,2433.70,"0,99%"
2041,2017-08-21,2428.40,2425.50,2430.60,2417.30,"0,12%"
2042,2017-08-18,2425.60,2427.60,2440.30,2420.70,"-0,18%"
2043,2017-08-17,2430.00,2462.90,2465.00,2430.00,"-1,54%"


In [13]:
OHLC_SPX[["SPX-Close", "SPX-Open", "SPX-High", "SPX-Low"]] = OHLC_SPX[["SPX-Close", "SPX-Open", "SPX-High", "SPX-Low"]].astype(float)
OHLC_SPX.dtypes

Date         datetime64[ns]
SPX-Close           float64
SPX-Open            float64
SPX-High            float64
SPX-Low             float64
SPX-% var            object
dtype: object

In [14]:
OHLC_SPX

,Date,SPX-Close,SPX-Open,SPX-High,SPX-Low,SPX-% var
0,2025-10-03,6715.79,6722.14,6750.87,6705.67,"0,01%"
1,2025-10-02,6715.35,6731.31,6731.94,6693.23,"0,06%"
2,2025-10-01,6711.20,6664.92,6718.48,6656.20,"0,34%"
3,2025-09-30,6688.46,6656.19,6691.25,6641.00,"0,41%"
4,2025-09-29,6661.21,6661.58,6677.31,6644.49,"0,26%"
...,...,...,...,...,...,...
2040,2017-08-22,2452.50,2433.80,2454.80,2433.70,"0,99%"
2041,2017-08-21,2428.40,2425.50,2430.60,2417.30,"0,12%"
2042,2017-08-18,2425.60,2427.60,2440.30,2420.70,"-0,18%"
2043,2017-08-17,2430.00,2462.90,2465.00,2430.00,"-1,54%"


# OHLC del XAU/USD (precio del oro en USD) desde la fecha más vieja en la que pudimos obtener datos del BTC
https://es.investing.com/currencies/xau-usd-historical-data

In [15]:
OHLC_XAUUSD = pd.read_csv("CSVs/XAU-USD.csv")
OHLC_XAUUSD = format_data(OHLC_XAUUSD, "XAUUSD")

OHLC_XAUUSD = OHLC_XAUUSD[['Date', 'XAUUSD-Close', 'XAUUSD-Open', 'XAUUSD-High', 'XAUUSD-Low', 'XAUUSD-% var']]
for feature in OHLC_XAUUSD.columns.values[1:-1]:
    OHLC_XAUUSD[feature] = OHLC_XAUUSD[feature].str.replace(".","")
    OHLC_XAUUSD[feature] = OHLC_XAUUSD[feature].str.replace(",",".")
OHLC_XAUUSD

,Date,XAUUSD-Close,XAUUSD-Open,XAUUSD-High,XAUUSD-Low,XAUUSD-% var
0,2025-10-05,3918.14,3889.33,3920.32,3884.49,"0,81%"
1,2025-10-03,3886.83,3857.92,3891.85,3838.05,"0,79%"
2,2025-10-02,3856.53,3866.66,3897.20,3819.51,"-0,24%"
3,2025-10-01,3865.80,3858.84,3895.45,3853.44,"0,19%"
4,2025-09-30,3858.51,3833.84,3871.87,3793.18,"0,64%"
...,...,...,...,...,...,...
2112,2017-08-22,1284.72,1291.18,1292.69,1282.11,"-0,50%"
2113,2017-08-21,1291.22,1283.60,1293.58,1280.60,"0,52%"
2114,2017-08-18,1284.50,1288.20,1301.20,1283.64,"-0,27%"
2115,2017-08-17,1288.01,1283.61,1290.47,1282.20,"0,38%"


In [16]:
OHLC_XAUUSD[["XAUUSD-Close", "XAUUSD-Open", "XAUUSD-High", "XAUUSD-Low"]] = OHLC_XAUUSD[["XAUUSD-Close", "XAUUSD-Open", "XAUUSD-High", "XAUUSD-Low"]].astype(float)
OHLC_XAUUSD.dtypes

Date            datetime64[ns]
XAUUSD-Close           float64
XAUUSD-Open            float64
XAUUSD-High            float64
XAUUSD-Low             float64
XAUUSD-% var            object
dtype: object

In [17]:
OHLC_XAUUSD

,Date,XAUUSD-Close,XAUUSD-Open,XAUUSD-High,XAUUSD-Low,XAUUSD-% var
0,2025-10-05,3918.14,3889.33,3920.32,3884.49,"0,81%"
1,2025-10-03,3886.83,3857.92,3891.85,3838.05,"0,79%"
2,2025-10-02,3856.53,3866.66,3897.20,3819.51,"-0,24%"
3,2025-10-01,3865.80,3858.84,3895.45,3853.44,"0,19%"
4,2025-09-30,3858.51,3833.84,3871.87,3793.18,"0,64%"
...,...,...,...,...,...,...
2112,2017-08-22,1284.72,1291.18,1292.69,1282.11,"-0,50%"
2113,2017-08-21,1291.22,1283.60,1293.58,1280.60,"0,52%"
2114,2017-08-18,1284.50,1288.20,1301.20,1283.64,"-0,27%"
2115,2017-08-17,1288.01,1283.61,1290.47,1282.20,"0,38%"


# Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUCSL) 
Buscamos el CPI para consumidores urbanos en lo que respecta a todos los artículos (de Estados Unidos), que es el índice de precios de una canasta de bienes y servicios pagados por consumidores urbanos, cuyos cambios porcentuales miden la tasa de inflación entre cualquier intervalo de tiempo.
Este índice tiene relación con el bitcoin porque si los precios de los bienes y servicios suben, el dinero pierde valor, entonces, si el IPC es muy alto, más personas pueden querer comprar BTC para asegurar su dinero y así su precio sube (esto no está garantizado).
Este índice incluye aproximadamente al 88% de la población estadounidense, y se basa en precios de comida, ropa, alojamiento, combustibles, tarifas de transporte, tarifas de servicios (agua, alcantarillado, impuestos de ventas).

https://fred.stlouisfed.org/series/CPIAUCSL

In [18]:
CPIAUCSL = pd.read_csv("CSVs/CPIAUCSL to 8.2025.csv")
CPIAUCSL.dtypes

observation_date     object
CPIAUCSL            float64
dtype: object

In [19]:
CPIAUCSL["observation_date"] = pd.to_datetime(CPIAUCSL["observation_date"], format="%Y-%m-%d")
CPIAUCSL.dtypes

observation_date    datetime64[ns]
CPIAUCSL                   float64
dtype: object

In [20]:
CPIAUCSL = CPIAUCSL.rename(columns={'observation_date':'Date'})
CPIAUCSL

,Date,CPIAUCSL
0,2017-08-01,245.183
1,2017-09-01,246.435
2,2017-10-01,246.626
3,2017-11-01,247.284
4,2017-12-01,247.805
...,...,...
92,2025-04-01,320.321
93,2025-05-01,320.580
94,2025-06-01,321.500
95,2025-07-01,322.132


# CBOE Volatility Index (VIX)
El índice de volatilidad CBOE (VIX) mide la expectativa del mercado sobre la futura volatilidad del índice S&P 500. Cuando el VIX sube, por lo general indica que los inversores esperan una mayor turbulencia en el mercado, lo cual puede estar asociado a caídas significativas o incertidumbre sobre el mismo, mientras que si baja, se suele asociar a un período de estabilidad.
Decidimos incluirlo porque en principio el VIX y el valor del BTC se creía que eran totalmente independientes uno del otro, pero luego de un tiempo se han observado patrones en los que se parece encontrar una cierta correlación negativa entre ambos.

https://es.investing.com/indices/volatility-s-p-500

In [21]:
VIX = pd.read_csv("CSVs/VIX.csv")
VIX.dtypes

Fecha        object
Último       object
Apertura     object
Máximo       object
Mínimo       object
Vol.        float64
% var.       object
dtype: object

In [22]:
VIX = format_data(VIX, "VIX")
VIX = VIX[["Date","VIX-Close","VIX-Open","VIX-High","VIX-Low","VIX-% var"]]
VIX

,Date,VIX-Close,VIX-Open,VIX-High,VIX-Low,VIX-% var
0,2025-10-03,"16,65","16,35","17,06","16,20","0,12%"
1,2025-10-02,"16,63","16,12","16,92","15,93","2,09%"
2,2025-10-01,"16,29","17,28","17,28","15,98","0,06%"
3,2025-09-30,"16,28","16,49","16,70","16,02","0,99%"
4,2025-09-29,"16,12","15,84","16,29","15,74","5,43%"
...,...,...,...,...,...,...
2078,2017-08-22,"11,35","12,60","12,94","11,35","-13,95%"
2079,2017-08-21,"13,19","14,59","14,74","13,07","-7,50%"
2080,2017-08-18,"14,26","15,38","16,04","13,32","-8,30%"
2081,2017-08-17,"15,55","11,81","15,77","11,54","32,45%"


In [23]:
for feature in VIX.columns.values[1:-1]:
    VIX[feature] = VIX[feature].str.replace(",",".")
VIX[["VIX-Close","VIX-Open","VIX-High","VIX-Low"]] = VIX[["VIX-Close","VIX-Open","VIX-High","VIX-Low"]].astype(float)
VIX.dtypes

Date         datetime64[ns]
VIX-Close           float64
VIX-Open            float64
VIX-High            float64
VIX-Low             float64
VIX-% var            object
dtype: object

In [24]:
VIX

,Date,VIX-Close,VIX-Open,VIX-High,VIX-Low,VIX-% var
0,2025-10-03,16.65,16.35,17.06,16.20,"0,12%"
1,2025-10-02,16.63,16.12,16.92,15.93,"2,09%"
2,2025-10-01,16.29,17.28,17.28,15.98,"0,06%"
3,2025-09-30,16.28,16.49,16.70,16.02,"0,99%"
4,2025-09-29,16.12,15.84,16.29,15.74,"5,43%"
...,...,...,...,...,...,...
2078,2017-08-22,11.35,12.60,12.94,11.35,"-13,95%"
2079,2017-08-21,13.19,14.59,14.74,13.07,"-7,50%"
2080,2017-08-18,14.26,15.38,16.04,13.32,"-8,30%"
2081,2017-08-17,15.55,11.81,15.77,11.54,"32,45%"


In [25]:
dataset=pd.merge(OHLC_WTI, OHLC_BRENT, on='Date', how='inner').merge(OHLC_SPX, on='Date', how='inner').merge(OHLC_XAUUSD, on='Date', how='inner').merge(VIX, on='Date', how='inner')

In [26]:
dataset.dtypes

Date            datetime64[ns]
WTI-Close              float64
WTI-Open               float64
WTI-High               float64
WTI-Low                float64
WTI-Volume             float64
WTI-% var               object
BRENT-Close            float64
BRENT-Open             float64
BRENT-High             float64
BRENT-Low              float64
BRENT-Volume           float64
BRENT-% var             object
SPX-Close              float64
SPX-Open               float64
SPX-High               float64
SPX-Low                float64
SPX-% var               object
XAUUSD-Close           float64
XAUUSD-Open            float64
XAUUSD-High            float64
XAUUSD-Low             float64
XAUUSD-% var            object
VIX-Close              float64
VIX-Open               float64
VIX-High               float64
VIX-Low                float64
VIX-% var               object
dtype: object

In [27]:
dataset
dataset = dataset.drop(columns=[col for col in dataset.columns if "% var" in col])
dataset.dtypes

Date            datetime64[ns]
WTI-Close              float64
WTI-Open               float64
WTI-High               float64
WTI-Low                float64
WTI-Volume             float64
BRENT-Close            float64
BRENT-Open             float64
BRENT-High             float64
BRENT-Low              float64
BRENT-Volume           float64
SPX-Close              float64
SPX-Open               float64
SPX-High               float64
SPX-Low                float64
XAUUSD-Close           float64
XAUUSD-Open            float64
XAUUSD-High            float64
XAUUSD-Low             float64
VIX-Close              float64
VIX-Open               float64
VIX-High               float64
VIX-Low                float64
dtype: object

In [28]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2, include_bias=False)
numeric_dataset = dataset.drop(columns=["Date"]).copy()
poly_array = poly.fit_transform(numeric_dataset)
poly_features = poly.get_feature_names_out(numeric_dataset.columns)

poly_df = pd.DataFrame(poly_array, columns=poly_features, index=numeric_dataset.index)

poly_df['Date'] = dataset['Date'].values

cols = ['Date'] + [c for c in poly_df.columns if c != 'Date']
poly_df = poly_df[cols]

poly_df

,Date,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,...,VIX-Close^2,VIX-Close VIX-Open,VIX-Close VIX-High,VIX-Close VIX-Low,VIX-Open^2,VIX-Open VIX-High,VIX-Open VIX-Low,VIX-High^2,VIX-High VIX-Low,VIX-Low^2
0,2025-10-03,60.88,60.70,61.38,60.55,209160.0,64.53,64.38,65.02,64.20,...,277.2225,272.2275,284.0490,269.7300,267.3225,278.9310,264.8700,291.0436,276.3720,262.4400
1,2025-10-02,60.48,61.78,62.54,60.40,290510.0,64.11,65.45,66.15,64.00,...,276.5569,268.0756,281.3796,264.9159,259.8544,272.7504,256.7916,286.2864,269.5356,253.7649
2,2025-10-01,61.78,62.46,62.89,61.40,274340.0,65.35,66.18,66.57,65.05,...,265.3641,281.4912,281.4912,260.3142,298.5984,298.5984,276.1344,298.5984,276.1344,255.3604
3,2025-09-30,62.37,63.14,63.26,62.03,271650.0,67.02,67.50,67.71,66.84,...,265.0384,268.4572,271.8760,260.8056,271.9201,275.3830,264.1698,278.8900,267.5340,256.6404
4,2025-09-29,63.45,65.07,65.40,62.98,294290.0,67.97,69.50,69.91,67.52,...,259.8544,255.3408,262.5948,253.7288,250.9056,258.0336,249.3216,265.3641,256.4046,247.7476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,2017-08-22,47.64,47.45,48.03,47.20,38400.0,51.58,51.37,51.93,51.08,...,128.8225,143.0100,146.8690,128.8225,158.7600,163.0440,143.0100,167.4436,146.8690,128.8225
2041,2017-08-21,47.37,48.72,48.75,47.03,129190.0,51.28,52.63,52.67,50.96,...,173.9761,192.4421,194.4206,172.3933,212.8681,215.0566,190.6913,217.2676,192.6518,170.8249
2042,2017-08-18,48.51,46.93,48.74,46.78,253070.0,52.41,50.61,52.64,50.54,...,203.3476,219.3188,228.7304,189.9432,236.5444,246.6952,204.8616,257.2816,213.6528,177.4224
2043,2017-08-17,47.09,46.80,47.19,46.46,608310.0,50.75,50.22,50.84,49.81,...,241.8025,183.6455,245.2235,179.4470,139.4761,186.2437,136.2874,248.6929,181.9858,133.1716


In [29]:
poly_df = pd.merge(poly_df, OHLC_BTC, on="Date", how="inner")
poly_df

,Date,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,...,VIX-Open VIX-High,VIX-Open VIX-Low,VIX-High^2,VIX-High VIX-Low,VIX-Low^2,BTC-Open,BTC-High,BTC-Low,BTC-Close,BTC-Volume
0,2025-10-03,60.88,60.70,61.38,60.55,209160.0,64.53,64.38,65.02,64.20,...,278.9310,264.8700,291.0436,276.3720,262.4400,122232.21,122800.00,121510.00,122391.00,8208.166780
1,2025-10-02,60.48,61.78,62.54,60.40,290510.0,64.11,65.45,66.15,64.00,...,272.7504,256.7916,286.2864,269.5356,253.7649,120529.35,123894.99,119248.30,122232.00,23936.328000
2,2025-10-01,61.78,62.46,62.89,61.40,274340.0,65.35,66.18,66.57,65.05,...,298.5984,276.1344,298.5984,276.1344,255.3604,118594.99,121022.07,118279.31,120529.35,19670.835030
3,2025-09-30,62.37,63.14,63.26,62.03,271650.0,67.02,67.50,67.71,66.84,...,275.3830,264.1698,278.8900,267.5340,256.6404,114048.94,118649.10,113966.67,118594.99,20036.395160
4,2025-09-29,63.45,65.07,65.40,62.98,294290.0,67.97,69.50,69.91,67.52,...,258.0336,249.3216,265.3641,256.4046,247.7476,114311.97,114792.00,112656.27,114048.93,15044.156330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,2017-08-22,47.64,47.45,48.03,47.20,38400.0,51.58,51.37,51.93,51.08,...,163.0440,143.0100,167.4436,146.8690,128.8225,4040.00,4265.80,4013.89,4114.01,1001.136565
2041,2017-08-21,47.37,48.72,48.75,47.03,129190.0,51.28,52.63,52.67,50.96,...,215.0566,190.6913,217.2676,192.6518,170.8249,4016.00,4104.82,3400.00,4040.00,966.684858
2042,2017-08-18,48.51,46.93,48.74,46.78,253070.0,52.41,50.61,52.64,50.54,...,246.6952,204.8616,257.2816,213.6528,177.4224,4108.37,4184.69,3850.00,4139.98,381.309763
2043,2017-08-17,47.09,46.80,47.19,46.46,608310.0,50.75,50.22,50.84,49.81,...,186.2437,136.2874,248.6929,181.9858,133.1716,4285.08,4371.52,3938.77,4108.37,1199.888264


In [30]:
poly_df["BTC-Close_next_day"] = poly_df["BTC-Close"].shift(1)
poly_df = poly_df.dropna(subset=["BTC-Close_next_day"]).copy()
poly_df

,Date,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,...,VIX-Open VIX-Low,VIX-High^2,VIX-High VIX-Low,VIX-Low^2,BTC-Open,BTC-High,BTC-Low,BTC-Close,BTC-Volume,BTC-Close_next_day
1,2025-10-02,60.48,61.78,62.54,60.40,290510.0,64.11,65.45,66.15,64.00,...,256.7916,286.2864,269.5356,253.7649,120529.35,123894.99,119248.30,122232.00,23936.328000,122391.00
2,2025-10-01,61.78,62.46,62.89,61.40,274340.0,65.35,66.18,66.57,65.05,...,276.1344,298.5984,276.1344,255.3604,118594.99,121022.07,118279.31,120529.35,19670.835030,122232.00
3,2025-09-30,62.37,63.14,63.26,62.03,271650.0,67.02,67.50,67.71,66.84,...,264.1698,278.8900,267.5340,256.6404,114048.94,118649.10,113966.67,118594.99,20036.395160,120529.35
4,2025-09-29,63.45,65.07,65.40,62.98,294290.0,67.97,69.50,69.91,67.52,...,249.3216,265.3641,256.4046,247.7476,114311.97,114792.00,112656.27,114048.93,15044.156330,118594.99
5,2025-09-26,65.72,65.20,66.42,64.66,284990.0,70.13,69.54,70.76,69.11,...,258.2481,290.7025,260.6945,233.7841,109643.46,109743.91,109064.40,109635.85,5501.786430,114048.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,2017-08-22,47.64,47.45,48.03,47.20,38400.0,51.58,51.37,51.93,51.08,...,143.0100,167.4436,146.8690,128.8225,4040.00,4265.80,4013.89,4114.01,1001.136565,4316.01
2041,2017-08-21,47.37,48.72,48.75,47.03,129190.0,51.28,52.63,52.67,50.96,...,190.6913,217.2676,192.6518,170.8249,4016.00,4104.82,3400.00,4040.00,966.684858,4114.01
2042,2017-08-18,48.51,46.93,48.74,46.78,253070.0,52.41,50.61,52.64,50.54,...,204.8616,257.2816,213.6528,177.4224,4108.37,4184.69,3850.00,4139.98,381.309763,4040.00
2043,2017-08-17,47.09,46.80,47.19,46.46,608310.0,50.75,50.22,50.84,49.81,...,136.2874,248.6929,181.9858,133.1716,4285.08,4371.52,3938.77,4108.37,1199.888264,4139.98


In [31]:
# DEFINIMOS TARGET Y FEATURES

poly_df["Day_Of_Week"] = poly_df["Date"].dt.dayofweek     # 0=Lunes, 6=Domingo
poly_df["Month"] = poly_df["Date"].dt.month       
poly_df["Year"] = poly_df["Date"].dt.year
poly_df = poly_df.drop('Date', axis=1)
poly_df["BTC_Close_MA7"] = poly_df["BTC-Close"].rolling(7).mean()
poly_df["BTC_Close_MA30"] = poly_df["BTC-Close"].rolling(30).mean()
poly_df["BTC_Close_diff1"] = poly_df["BTC-Close"].diff(1)
poly_df

,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,BRENT-Volume,...,BTC-Low,BTC-Close,BTC-Volume,BTC-Close_next_day,Day_Of_Week,Month,Year,BTC_Close_MA7,BTC_Close_MA30,BTC_Close_diff1
1,60.48,61.78,62.54,60.40,290510.0,64.11,65.45,66.15,64.00,480220.0,...,119248.30,122232.00,23936.328000,122391.00,3,10,2025,NaN,NaN,NaN
2,61.78,62.46,62.89,61.40,274340.0,65.35,66.18,66.57,65.05,420600.0,...,118279.31,120529.35,19670.835030,122232.00,2,10,2025,NaN,NaN,-1702.65
3,62.37,63.14,63.26,62.03,271650.0,67.02,67.50,67.71,66.84,19230.0,...,113966.67,118594.99,20036.395160,120529.35,1,9,2025,NaN,NaN,-1934.36
4,63.45,65.07,65.40,62.98,294290.0,67.97,69.50,69.91,67.52,107590.0,...,112656.27,114048.93,15044.156330,118594.99,0,9,2025,NaN,NaN,-4546.06
5,65.72,65.20,66.42,64.66,284990.0,70.13,69.54,70.76,69.11,110380.0,...,109064.40,109635.85,5501.786430,114048.93,4,9,2025,NaN,NaN,-4413.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,47.64,47.45,48.03,47.20,38400.0,51.58,51.37,51.93,51.08,221020.0,...,4013.89,4114.01,1001.136565,4316.01,1,8,2017,4416.521429,4163.520333,-202.00
2041,47.37,48.72,48.75,47.03,129190.0,51.28,52.63,52.67,50.96,172770.0,...,3400.00,4040.00,966.684858,4114.01,0,8,2017,4318.680000,4157.900667,-74.01
2042,48.51,46.93,48.74,46.78,253070.0,52.41,50.61,52.64,50.54,186530.0,...,3850.00,4139.98,381.309763,4040.00,4,8,2017,4259.371429,4152.233333,99.98
2043,47.09,46.80,47.19,46.46,608310.0,50.75,50.22,50.84,49.81,231590.0,...,3938.77,4108.37,1199.888264,4139.98,3,8,2017,4190.927143,4143.228667,-31.61


In [32]:
X = poly_df.copy()
X = X.dropna()
y = X['BTC-Close_next_day']
y_last_row = X.head(1)['BTC-Close_next_day'].copy()
X = X.drop('BTC-Close_next_day', axis=1)
X

,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,BRENT-Volume,...,BTC-High,BTC-Low,BTC-Close,BTC-Volume,Day_Of_Week,Month,Year,BTC_Close_MA7,BTC_Close_MA30,BTC_Close_diff1
30,63.52,62.85,63.67,62.52,250170.0,67.12,66.46,67.25,66.25,275960.0,...,117429.05,111684.79,116935.99,23128.090370,3,8,2025,112165.700000,113733.139667,1497.94
31,63.21,62.60,63.55,62.39,27160.0,66.31,65.59,66.54,65.31,263660.0,...,114821.76,112015.67,112500.00,10839.692350,2,8,2025,112691.938571,113408.739667,-4435.99
32,62.35,63.27,63.39,62.25,99480.0,65.30,65.98,66.01,65.14,200650.0,...,114615.38,112380.00,114271.24,15636.341940,1,8,2025,113533.915714,113200.136000,1771.24
33,63.42,63.00,63.79,62.18,95110.0,66.01,65.26,66.24,64.79,195570.0,...,116725.69,112732.58,112872.94,18065.474240,0,8,2025,113577.635714,113009.401000,-1398.30
34,62.80,63.91,64.15,62.68,197390.0,65.85,66.87,67.06,65.73,251200.0,...,117898.99,117143.98,117380.66,6393.681170,4,8,2025,114451.728571,113120.458667,4507.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,47.64,47.45,48.03,47.20,38400.0,51.58,51.37,51.93,51.08,221020.0,...,4265.80,4013.89,4114.01,1001.136565,1,8,2017,4416.521429,4163.520333,-202.00
2041,47.37,48.72,48.75,47.03,129190.0,51.28,52.63,52.67,50.96,172770.0,...,4104.82,3400.00,4040.00,966.684858,0,8,2017,4318.680000,4157.900667,-74.01
2042,48.51,46.93,48.74,46.78,253070.0,52.41,50.61,52.64,50.54,186530.0,...,4184.69,3850.00,4139.98,381.309763,4,8,2017,4259.371429,4152.233333,99.98
2043,47.09,46.80,47.19,46.46,608310.0,50.75,50.22,50.84,49.81,231590.0,...,4371.52,3938.77,4108.37,1199.888264,3,8,2017,4190.927143,4143.228667,-31.61


In [33]:
x_last_row = X.head(1).copy()
X = X.iloc[:-1]
y = y.iloc[:-1]
x_last_row

,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,BRENT-Volume,...,BTC-High,BTC-Low,BTC-Close,BTC-Volume,Day_Of_Week,Month,Year,BTC_Close_MA7,BTC_Close_MA30,BTC_Close_diff1
30,63.52,62.85,63.67,62.52,250170.0,67.12,66.46,67.25,66.25,275960.0,...,117429.05,111684.79,116935.99,23128.09037,3,8,2025,112165.7,113733.139667,1497.94


In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# DIVIDIMOS EL DATASET 70% TRAIN 15% TEST 15% VALIDATION
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=False
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, shuffle=False
)

# PREPROCESAMIENTO
num_cols = [c for c in X_train.columns] #if c != "Date"
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols)
    ],
)

# PIPELINE
rf_pipeline = Pipeline(steps=[
    ("scaler", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=200,
        max_depth=None,
        random_state=42,
        n_jobs=-1
    ))
])

# ENTRENAMOS
rf_pipeline.fit(X_train, y_train)

# EVALUAMOS
y_val_pred = rf_pipeline.predict(X_val)
mae_val = mean_absolute_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)
print("1- RANDOM FOREST VALIDATION RESULTS")
print(f"MAE: {mae_val:.4f}")
print(f"R2: {r2_val:.4f}")

y_test_pred = rf_pipeline.predict(X_test)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
print("2- RANDOM FOREST TEST RESULTS")
print(f"MAE: {mae_test:.4f}")
print(f"R2: {r2_test:.4f}")


1- RANDOM FOREST VALIDATION RESULTS
MAE: 1540.0923
R2: 0.3516
2- RANDOM FOREST TEST RESULTS
MAE: 1073.5787
R2: 0.7560


In [35]:
X_train_scaled = rf_pipeline.named_steps["scaler"].transform(X_train)

scaled_feature_names = num_cols  
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=scaled_feature_names)

X_train_scaled_df.head()

,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,BRENT-Volume,...,BTC-High,BTC-Low,BTC-Close,BTC-Volume,Day_Of_Week,Month,Year,BTC_Close_MA7,BTC_Close_MA30,BTC_Close_diff1
0,-0.373080,-0.410913,-0.426844,-0.363287,0.243951,-0.379642,-0.414643,-0.429155,-0.365635,0.100872,...,2.501826,2.449537,2.554252,-0.615207,0.699766,0.492344,1.625899,2.373913,2.375201,0.928547
1,-0.390084,-0.424620,-0.433323,-0.370541,-1.370779,-0.421683,-0.459810,-0.465403,-0.415314,0.001725,...,2.412164,2.461294,2.399367,-0.747422,-0.016782,0.492344,1.625899,2.392240,2.364016,-2.572918
2,-0.437255,-0.387885,-0.441960,-0.378352,-0.847138,-0.474104,-0.439562,-0.492462,-0.424299,-0.506179,...,2.405067,2.474240,2.461211,-0.695813,-0.733330,0.492344,1.625899,2.421562,2.356823,1.089815
3,-0.378565,-0.402688,-0.420366,-0.382258,-0.878779,-0.437253,-0.476942,-0.480719,-0.442797,-0.547127,...,2.477638,2.486768,2.412389,-0.669677,-1.449879,0.492344,1.625899,2.423085,2.350247,-0.780452
4,-0.412573,-0.352794,-0.400932,-0.354360,-0.138209,-0.445558,-0.393357,-0.438855,-0.393117,-0.098711,...,2.517987,2.643516,2.569778,-0.795258,1.416314,0.492344,1.625899,2.453526,2.354076,2.704544


In [36]:
x_last_row['Day_Of_Week'] = x_last_row['Day_Of_Week']
x_last_row

,WTI-Close,WTI-Open,WTI-High,WTI-Low,WTI-Volume,BRENT-Close,BRENT-Open,BRENT-High,BRENT-Low,BRENT-Volume,...,BTC-High,BTC-Low,BTC-Close,BTC-Volume,Day_Of_Week,Month,Year,BTC_Close_MA7,BTC_Close_MA30,BTC_Close_diff1
30,63.52,62.85,63.67,62.52,250170.0,67.12,66.46,67.25,66.25,275960.0,...,117429.05,111684.79,116935.99,23128.09037,3,8,2025,112165.7,113733.139667,1497.94


In [37]:
y_last_row

30    115438.05
Name: BTC-Close_next_day, dtype: float64

In [38]:
y_predicted_last_row = rf_pipeline.predict(x_last_row)
y_predicted_last_row

array([115453.32475])

In [39]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(
    hidden_layer_sizes=(128, 64),
    max_iter=3000,
    random_state=42
)

In [ ]:
mlp_pipeline = Pipeline(steps=[
    ("scaler", preprocessor),
    ("model", mlp)
])

mlp_pipeline.fit(X_train, y_train)

y_mlp_val = mlp_pipeline.predict(X_val)
mlp_mae_val = mean_absolute_error(y_val, y_mlp_val)
mlp_r2_val = r2_score(y_val, y_val_pred)
print("1- MLP VALIDATION RESULTS")
print(f"MAE: {mlp_mae_val:.4f}")
print(f"R2: {mlp_r2_val:.4f}")

y_mlp_test = mlp_pipeline.predict(X_test)
mlp_mae_test = mean_absolute_error(y_test, y_mlp_test)
mlp_r2_test = r2_score(y_test, y_mlp_test)
print("2- MLP TEST RESULTS")
print(f"MAE: {mlp_mae_test:.4f}")
print(f"R2: {mlp_r2_test:.4f}")

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score

xgb = XGBRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.5,
    reg_alpha=0.5,
    min_child_weight=3,
    random_state=42,
    eval_metric="mae",
    early_stopping_rounds=50
)

xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)



y_pred_test = xgb.predict(X_test)
y_pred_val = xgb.predict(X_val)

print("1- XGBOOST VALIDATION RESULTS")
print(f"MAE: {mean_absolute_error(y_val, y_pred_val):.4f}")
print(f"R2: {r2_score(y_val, y_pred_val):.4f}")

print("2- XGBOOST TEST RESULTS")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.4f}")
print(f"R2: {r2_score(y_test, y_pred_test):.4f}")